# Solid Waste Quick Assessment Tool

## [Jump to Walkthrough](#walkthrough)

> Quickly compute, and display on a map, the amount of solid waste generated per week per municipal jurisdiction, and the amount of uncollected solid waste per service area. Applicable to any of the around 30+ countries covered by the [HRSL](https://ciesin.columbia.edu/data/hrsl/#data) at 1 arc-second resolution (2015 data) or globally at 30 arc-second resolution through the [GPW](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-rev11/data-download) raster (2000, 2005, 2010, 2015 and 2020 data).   

This script will compute two statistics (both in metric tonnes): the amount of solid waste generated per week per municipal jurisdiction, and the amount of uncollected solid waste generated in each 'service area', i.e. the area of a municipal jurisdiction for which a service provider (e.g. contractor or municipal department) provides solid waste collection services. 

Both statistics are presented in a rank-ordered list and as a choropleth map. The example below shows the outputs for Lagos State in Nigeria, using dummy data with respect to service areas.

**Results updated in real-time:** A **drop-down menu** allows for the superordinate jurisdiction (e.g. the state-level) to be selected. A **slider** allows for assumptions on the amount of solid waste generated per capita per day (in kilograms) to be adjusted interactively. Both lists and maps are then updated in real-time.

![Screen capture of outputs 1 and 2](output1and2.png)
![Screen capture of output 3](output3.png)

## How it works

The script performs simple algebra on the pixel values of a raster layer representing population estimates and adds them as zonal statistics to new attribute fields for two sets of polygon features before plotting each on a map.
Each step in the calculation is modularised as a sub-task in a sequence of functions, as pictured below.

![Process diagram](process.png)

## Installation

If you are new to Python, the easiest way to run the script on a specific jurisdiction would be to first install a scientific Python distribution like [Anaconda](https://docs.anaconda.com/anaconda/install/). Anaconda comes with 250 scientific and analytic so-called 'packages' such as NumPy, Pandas, SciPy, Matplotlib, and IPython pre-installed. The Anaconda [Cheat Sheet](https://docs.anaconda.com/_downloads/9ee215ff15fde24bf01791d719084950/Anaconda-Starter-Guide.pdf) offers a good 2-page introduction.

Anaconda includes the [conda](https://conda.io/en/latest/) command line interface as well the graphical user interface (GUI) **Navigator**.

> Anaconda is available for Windows 7 and newer, macOS 10.10 and newer, or any Linux distribution with a glibc version greater than 2.12 (CentOS 6). It requires 3GB of free hard drive space (Miniconda, a much smaller installer containing only Conda and its dependencies, needs only 400 MB). The environment required for this script (usually to then be found under `C:\User\YOURUSERNAME\anaconda3\env\` will take up an additional 1.9GB.

The following dependencies are required:

```sh
  - python=3.8.8
  - geopandas=0.9.0
  - cartopy=0.18.0
  - notebook=6.2.0
  - rasterio=1.2.0
  - rasterstats=0.14.0
  - ipywidgets=7.6.3
```

To ensure access to these packages and avoid [dependency hell](https://en.wikipedia.org/wiki/Dependency_hell), it is necessary to set up an _environment_, the requirements for which are contained in the `environment.yml` file in the root of this repository (or 'repo').

After [forking](https://docs.github.com/en/github/getting-started-with-github/fork-a-repo) this repo or downloading the .zip, open Navigator, click on the **Import** button at the bottom of the **Environments** tab, navigate to the `environment.yml` file in the root folder of your local repo, and click **Import**. Setting up the environment with all its packages and dependencies may take a few minutes.

## Running the script

#### Required source files

The script requires three source files:

1. a vector source in EPSG:4326 indicating the administrative boundaries for a superordinate sub-national government tier (e.g. the state level) and a second subordinate tier (e.g. the municipal level):
 > the present sample script uses the administrative boundaries for Nigeria available at [humdata.org](https://data.humdata.org/dataset/nga-administrative-boundaries);
2. another vector source in EPSG:4326 representing service areas including an attribute field or column indicating the total amount of solid waste collected per week by each contractor/service provider in metric tonnes:
 > the present sample script uses dummy polygons and hypothetical collection totals for Lagos State assuming a 21% collection rate. A second shapefile containing dummy data for Ogun State is available in the `data_files` folder as well. **Switch between Lagos and Ogun in the drop-down menu** to demonstrate how the script can be quickly run on any jurisdiction;       
3. the pop GeoTIFF of the CIESIN [High-Resolution Settlements Layer (HRSL)](https://ciesin.columbia.edu/data/hrsl/#data) providing the number of persons estimated to haved lived in each 1 arc-second pixel (roughly 30m) in 2015, available for roughly 30 countries in Africa, Asia and Latin America, **or** the [Gridded World Population](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4) layer which has global coverage, covers 5 year periods from 2000 to 2020 but only has a 30 arc-second resolution (roughly 1km).
 > the present sample script uses the HRSL for Nigeria available [here](https://ciesin.columbia.edu/data/hrsl/#data).

#### Required script adjustments

As part of the `main()` function definition, two default values need to be adjusted:

1. the `state_list` default value which indicates the state to be selected on-load. Setting this is useful when service area data sources are not available for all states, as in the case of the sample data. Selecting such a state, or having the first state in the list be selected automatically, will cause an `OpenFailedError`;
2. the `sw_ppd` _min_, _max_ and _step_ floats indicating the amount of solid waste generated per capita per day in kilograms, according to your particular context (the map annotation will be adjusted automatically).

Enclosing scope variables will need to be adjusted in the `USER INPUTS 1` section. These are:

3. the `mun_name_field` variable indicating the name of the column containing the names of municipalities;
4. the `fp_service_areas` variable indicating the file path to the service areas data sources - the sample data appends '_statename.shp' to 'service_areas' to construct file names;
5. the `provider_name_field` variable indicating the name of the column containing the names of service providers;
6. the `provider_coll_field` variable indicating the name of the column containing the weekly collection totals reported;
7. the `fp_raster` variable indicating the filepath to the raster data source;

Global scope variables will need to be adjusted in the `USER INPUTS 2` section, namely:

8. the `fp_adm` variable indicating the filepath to the administrative boundaries data source;
9. the `state_name_field` variable indicating the name of the column containing the names of the superordinate (e.g. state-level) jurisdictions. If this information is in a data source separate from the subordinate tier, a spatial join via GeoPandas or a desktop GIS may be necessary.

#### Optional customization

Changing the `stat_select` variable to another statistic supported by _rasterstats_ (such as min, max, mean etc.) is also possible, though then the array algebra and the title of the choropleth maps (via `var_name`) will need to be adjusted accordingly.  

#### How to run the script from inside an IDE or text editor

To more fluidly edit the script and see the results side-by-side in the same window, you may want to consider using an IDE or code editor with a Python extension. A number of options like [PyCharm](https://www.jetbrains.com/pycharm/) or [Spyder](https://www.spyder-ide.org/) are available, with [Visual Studio Code's Python Extension](https://code.visualstudio.com/docs/languages/python) pictured below. 

![Screenshot of the Visual Studio Code Python Extension](instructions_02.png)

Adding `# %%` to the top of the script turns the script into a [Jupyter-like code cell](https://code.visualstudio.com/docs/python/jupyter-support-py) which allows for a seamless workflow of editing and results visualization in the same window. Simply edit the script on the left and hit `Shift + Enter`to execute and display the results in the interactive interpreter on the right.

#### Expected Outputs

The script will return three outputs:
1. a list of municipalities by amount of solid waste generated per week in metric tonnes, in descending order;
2. a list of service providers by amount of uncollected solid waste per week in metric tonnes, in descending order;
3. choropleth maps visualizing each list using a scalar colormap (blues for (1) and reds for (2)).

A drop-down menu will allow for selecting superordinate jurisdictions whereas a slider will allow for adjusting assumptions on the amount of solid waste generated per capita per day in kilograms.

## Release History

* 0.1.0
    * first release

## Meta

Gregor Herda – gregorherda at gmail.com

The Solid Waste Quick Assessment Tool is licensed under the terms of the GNU General Public License v3.0 and is available for free. See ``LICENSE`` for more information.

[https://github.com/gregorhuh](https://github.com/gregorhuh)

## How to Contribute

1. Fork it (<https://github.com/gregorhuh/UU_egm722_project/fork>)
2. Create your feature branch (`git checkout -b feature/fooBar`)
3. Commit your changes (`git commit -am 'Add some fooBar'`)
4. Push to the branch (`git push origin feature/fooBar`)
5. Create a new Pull Request with comprehensive description of changes

## Acknowledgements

* @iamdonovan for his assistance in fixing the affine argument




# Walkthrough<a id="walkthrough"></a>

Hit `Shift + Enter` in each of the following code cells to run the code.

#### Importing Modules

The script first imports necessary modules which are .py files containing Python scripts which can be executed by the Python interpreter directly.

In [12]:
import numpy as np
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
import rasterio as rio
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

%matplotlib inline

#### Overall structure

Subsequently the script’s four sub-task functions are defined as indicated by the **def** keyword which denotes a user defined function. The `main()` function constitutes the script’s entry point. As `main()` executes all sub-task functions it is defined last to make it easier to follow what's happening.


#### Sub-task function 1: `getVector()`

The first sub-task function called by `main()` is `getVector()` which converts the vector sources to geopandas GeoDataFrames whereby a subset of features is selected based on the name of the superordinate jurisdiction dynamically supplied by interact() as the state_list/state_select variable (the service areas are also turned into a GeoDataFrame, though in the sample data all features are inside the study area, so no subsetting is required).

In [13]:
def getVector(fp_adm, fp_service_areas, state_name_field, state_select='Lagos'):
    """Returns a subset of vector features and the bounding box (study area).

    Parameters
    ----------

    fp_adm : str
        File path for the administrative boundaries vector source.
    fp_service_areas : str
        File path for the service areas vector source.
    state_name_field : str
        The name of the column containing names of superordinate jurisdictions.
    state_select : str
        The name of the superordinate jurisdiction (state) for which to conduct the analysis.

    Returns
    ----------
    municipal_filter : GeoDataFrame
        GDF of the jurisdictions.
    bbox : ndarray
        Bounding box of the study area.
    service_areas : GeoDataFrame
        GDF of the service areas
    """

    # LOAD ADMINISTRATIVE BOUNDARIES VECTOR DATA

    # load Nigeria Local Government Area boundaries (Level 2, 'ADM2_EN'), and select only those LGAS within the larger Lagos State administrative boundary (Level 1, 'ADM1_EN')

    municipal_all = gpd.read_file(fp_adm)
    
    municipal_filter = municipal_all[municipal_all[state_name_field] == state_select]
    
    # Somehow feed to interact() for drop-down: state_list = sorted(municipal_all[state_name_field].unique().tolist())

    # DEFINE STUDY AREA BASED ON VECTOR SELECTION

    bbox = municipal_filter.total_bounds

    # LOAD VECTOR DATA FOR SERVICE AREAS OF SOLID WASTE SERVICE PROVIDERS

    # service_areas = gpd.read_file(fp_service_areas).to_crs(crs)
    service_areas = gpd.read_file(fp_service_areas)
    
    return municipal_filter, bbox, service_areas

#### Sub-task function 2: `computeArray()`

The subset then serves to define the bounding box or study area as an nd array (´bbox´), defined by the total bounds of the selected features.

Subsequently, as part of the ´computeArray()´ sub-task function, the GeoTIFF is read using *rasterio*’s `open()` method creating a DatasetReader object called `dataset`. 

Importantly, this is done as part of a **with** statement which ensures that the GeoTIFF is closed after the nested block which follows the **with** statement exits, regardless of how it exits.
`computeArray()` then unpacks  (`*bbox`) the elements in the nd array to serve as arguments to the DatasetReader’s `window()` method to construct a `Window` object. This object then allows for windowed reading of only the relevant part of the `dataset`  into a 2-d numpy array containing the pixel values of population estimates.

Simple algebra is then performed on the array to arrive at the weekly weight of solid waste generated in tonnes.
Besides assigning `dataset`’s `affine` attribute to the `affine` variable, the `nodata` attribute is also assigned to a variable. When computing zonal statistics, this later ensures that non-zero NoData values do not affect the computation.


In [14]:
def computeArray(fp_raster, bbox, sw_ppd):
    """Returns the nd array, affine and nodata variables required for zonal_stats.

    Parameters
    ----------
    fp_raster : str
        File path to the HRSL or GWP raster source.
    bbox : nd array
        Bounding box of the study area.
    sw_ppw : float
        The amount of solid waste generated per capita per day.
    
    Returns
    ----------
    array : nd array
        Array representing the amount of solid waste produce per pixel per week in metric tonnes.
    affine : Affine
        Affine transformation for the study area.
    nodata : type depends on raster source
        The NoData value of the raster source.
    """ 
    # 1. LOAD HIGH RESOLUTION SETTLEMENTS LAYER

    # Continuous floating point raster layer by CIESIN representing number of persons per 30x30m grid cell
    # Sample: Nigeria

    with rio.open(fp_raster) as dataset:
    
        # read CRS and no data attributes, create Window object
        crs = dataset.crs
        nodata = dataset.nodata
        window = dataset.window(*bbox)

        # CREATE NUMPY ND ARRAYS

        # load a subset of the HRSL corresponding to the study area
        pop_array = dataset.read(1, window=window)
        affine = dataset.window_transform(window)
        pop_array[(pop_array < 0)] = np.nan # sets negative NoData values to NaN to enable array algebra

        # Calculate tons of solid waste produced per grid cell rson per week

        sw_ppd_array = pop_array * sw_ppd # converts population to solid waste per person and day

        sw_ppw_array = sw_ppd_array * 7 # converts daily to weekly figures

        array = sw_ppw_array / 1000 # converts kilograms to tons per week (TPW)
        
        return array, affine, nodata


#### Sub-task function 3: `zonalStats()`

The subsequent `zonalStats()` sub-task function wraps the script’s most central function, `getNamesStats()`. This function is called twice (once for jurisdictions and once for service areas), itself wraps the *rasterstats* `zonal_stats()` function and populates two lists, one for feature names and one for features’ zonal statistics. It takes five positional and one keyword argument.

The arguments `vector`, `array` and `stat_select` are passed through to the `zonal_stats` function in the *rasterstats* module representing the vector source, the raster source and the statistic to be computed, respectively. The sum of pixel values will be computed by default.

Due to the optional keyword argument `geojson_out=True` (default is `False`) `zonal_stats(`) outputs a list of **dictionaries** containing the polygon features’ attributes in addition to a new attribute containing the zonal statistic. As part of `getNamesStats()` the positional arguments `name_field`, `feature_list` and `name_list` are passed in order to query the `properties` dictionary inside the `zonal_stats()` list output, here called `temp`, by way of a **for** loop and extract, for each polygon feature, its name and corresponding zonal statistic, appending it to two empty lists. As `getNamesStats()` is run twice, the use of the `feature_list` and `stat_list` arguments allows for results to be assigned to different lists during multiple runs of the function.

The `array` variable then serves as the raster argument for `getNamesStats()`. The `affine` and `nodata` arguments for `zonal_stats()` are supplied by `computeArray()`.


In [15]:
def zonalStats(municipal_filter, service_areas, array, affine, nodata, stat_select, mun_name_field, provider_name_field):
        """Returns one list each of feature names and zonal statistics.
        
        Parameters
        ----------
        municipal_filter : GeoDataFrame
            GDF of the jurisdictions.
        service_areas : GeoDataFrame
            GDF of the service areas
        array : nd array
            Array representing the amount of solid waste produce per pixel per week in metric tonnes.
        affine : Affine
            Affine transformation for the study area.
        nodata : type depends on raster source
            The NoData value of the raster source.
        stat_select : str
            The statistic to be computed (default is 'sum')
        mun_name_field : str
            The name of the column containing the names of subordinate jurisdictions.
        provider_name_field : str
            The name of the column containing the names of service providers.
        
        Returns
        ----------
        mun_names : list
            List of names of subordinate jurisdictions.
        mun_stats : list
            List of zonal statistics for subordinate jurisdictions.
        provider_names : list
            List of names of service providers
        provider_stats : list
            List of zonal statistics for service providers.
        """

        def getNamesStats(vector, array, name_field, feature_list, stat_list, stat_select='sum'):
            """Returns one list each of feature names and zonal statistics.

            Parameters
            ----------
            vector : path to a vector source or geo-like python object
                Python object can be a GeoPandas GeoDataFrame.
            raster : ndarray
                rasterstats alternative arg, 'path to a GDAL raster', not accepted.   
            name_field : str
                The vector source's column name containing feature names.
            feature_list : str
                Temp variable name for feature list. Must be unique in script as run consecutively.
            stat_list : str
                Temp variable name for statistics list. Must be unique in script.
            stat_select : str, optional
                String value for any statistic supported by zonal_stats (the default is 'sum').

            Returns
            ----------
            feature_list : list
                A list in alphabetical order containing the names of polygon features for which zonal stats are computed.
            stat_list : list
                A list containing the zonal statistics in the same order as "feature_list".
            """

            temp = zonal_stats(vector, array, affine=affine, nodata=nodata, stats=stat_select, geojson_out=True)

            for feature_dict in temp:
                feature_name = feature_dict['properties'][name_field]
                feature_stat = feature_dict['properties'][stat_select]
                feature_list.append(feature_name)
                stat_list.append(feature_stat)

        # CALCULATE ZONAL STATS - BASELINE GENERATION PER MUNICIPALITY

        # empty lists of all polygon names and zonal stats to be populated by function

        mun_names = []
        mun_stats = []
        getNamesStats(municipal_filter, array, mun_name_field, mun_names, mun_stats)

        # CALCULATE ZONAL STATS - SOLID WASTE COLLECTED PER SERVICE AREA

        provider_names = []
        provider_stats = []
        getNamesStats(service_areas, array, provider_name_field, provider_names, provider_stats)

        return mun_names, mun_stats, provider_names, provider_stats

#### Sub-task function 4: `processStats()`

As the statistic for service areas is the total amount of *un*collected waste, and total waste volume actually collected is provided as part of the `service_area` GeoDataFrame, as part of a final sub-task function `processStats()`, the latter values are first extracted to a **list**

```
provider_coll = service_areas[provider_coll_field].values.tolist()
```
which is subsequently turned into a `zip` object (an iterator aggregating the two iterables)

```
zip_object = zip(provider_stats, provider_coll)
```

which allows for the actual collected waste volume to be subtracted from the amount estimated to have been generated 
through a **for** loop and appended to a new **list**.

```
provider_uncoll = []
    for i, j in zip_object:
        provider_uncoll.append(i - j)
```

To print the results as a rank ordered **list**, the two list pairs are zipped and turned into a **dict**

```
mun_dict = dict(zip(mun_names, mun_stats))
provider_dict = dict(zip(provider_names, provider_uncoll))
```
which allows for the **dict** to be sorted by values through a `lambda` function inside a `sorted()` function.

```
mun_dict_sorted = sorted(mun_dict.items(), key=lambda x: x[1], reverse=True)
provider_dict_sorted = sorted(provider_dict.items(), key=lambda x: x[1], reverse=True) 
```
resulting in a **list** containing the rank ordered element pairs which can be printed successively through a **for** loop:

```
print('Municipalities by solid waste generated per week (descending):\n')
    for i in mun_dict_sorted:
        print(i[0],':',f"{int(i[1]):,}", 'tonnes')

print('\nService providers by total uncollected solid waste per week (descending)\n')
    for i in provider_dict_sorted:
        print(i[0],':',f"{int(i[1]):,}", 'tonnes')
```



In [16]:
def processStats(service_areas, mun_names, mun_stats, provider_names, provider_stats, provider_coll_field):
        """Prints two rank-ordered lists and returns two lists of ordered tuples.

        Parameters
        ----------
        service_areas : GeoDataFrame
            GDF of the service areas
        mun_names : list
            List of names of subordinate jurisdictions.
        mun_stats : list
            List of zonal statistics for subordinate jurisdictions.
        provider_names : list
            List of names of service providers
        provider_stats : list
            List of zonal statistics for service providers.
        provider_coll_field : str
            Name of the column containing the recorded collection totals for each service provider.

        Returns
        ----------
        mun_dict_sorted : list
            List of name-stat tuples for subordinate jurisdictions.
        provider_dict_sorted : list
            List of name-stat tuples for service providers.
        """

        # extract total collection values and subtract the total waste generated in each service area

        provider_coll = service_areas[provider_coll_field].values.tolist()
        zip_object = zip(provider_stats, provider_coll)

        provider_uncoll = []
        for i, j in zip_object:
            provider_uncoll.append(i - j)

        # ORGANISE AND PRINT RESULTS

        # combine populated lists into dictionaries
        mun_dict = dict(zip(mun_names, mun_stats))
        provider_dict = dict(zip(provider_names, provider_uncoll))

        # sort dictionaries by value in descending order into list of tuples
        mun_dict_sorted = sorted(mun_dict.items(), key=lambda x: x[1], reverse=True)
        provider_dict_sorted = sorted(provider_dict.items(), key=lambda x: x[1], reverse=True)

        # print the items in the sorted list of tuples
        print('Municipalities by solid waste generated per week (descending):\n')
        for i in mun_dict_sorted:
            print(i[0],':',f"{int(i[1]):,}", 'tonnes')

        print('\nService providers by total uncollected solid waste per week (descending)\n')
        for i in provider_dict_sorted:
            print(i[0],':',f"{int(i[1]):,}", 'tonnes')

        return mun_dict_sorted, provider_dict_sorted

#### The `main()` function

`main()` contains the **USER INPUTS 1** section. Here all hard coded variables with enclosing scope for which user input is required are listed for easy of access. Variables relate to file paths for service areas and the raster source, attribute names specific to the respective data source, and optional variables for the zonal statistic to be computed along with a variable adjusting the map title accordingly. As `main()` also executes all subsequent functions, this makes these variables accessible to subsequently called functions.

Before the results can be plotted on a map, they must be added back to the respective GeoDataFrames as a new column, which is done by `main()`. Two different approaches fulfilling the same purpose are used: the GeoDataFrame’s `assign()` method and a **for** loop. In the former, a `stat_output` column is created, in the latter a `total_uncoll` column. 

```
municipal_filter = municipal_filter.assign(
        stat_output = pd.Series(mun_stats, index = municipal_filter.index)
    )

for i, row in service_areas.iterrows():
        service_areas.loc[i, 'total_uncoll'] = provider_stats[i] - row[provider_coll_field]
```
According to the values in these columns, features are then plotted by *matplotlib* 

```
municipal_filter.plot(column='stat_output', cmap='Blues', linewidth=0.8, ax=ax1, edgecolor='0.8')
service_areas.plot(column='total_uncoll', cmap='Reds', linewidth=0.8, ax=ax2, edgecolor='k')
```
onto two separate subplots (ax1, ax2)

```
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7.5), subplot_kw=dict(projection=myCRS))
```

The respective colorbars are normalized through their minimum and maximum values

```
vmin, vmax =  municipal_filter['stat_output'].min(), municipal_filter['stat_output'].max()
vmin2, vmax2 =  service_areas['total_uncoll'].min(), service_areas['total_uncoll'].max()
```

assigning each a separate color map

```
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
fig.colorbar(sm, ax=ax1, orientation="horizontal")

sm.set_array([])
fig.colorbar(sm, ax=ax2, orientation="horizontal")
```

Certain keywords of both the map title and annotation are linked to the user inputs section.

```
title = var_name + ' by municipalities in ' + state_select + ' State'
ax1.set_title(title, fontdict={'fontsize': '12', 'fontweight' : '5'})
    
ax1.annotate('Source: CIESIN HRSL, assuming ' + str(sw_ppd) + 'kg of solid waste per capita per day', xy=(0.225, .025), xycoords='figure fraction', fontsize=12, color='#555555')
```


In [17]:
def main(state_list='Lagos', sw_ppd=(0.4, 1.2, 0.1)): 
    """Executes all functions, adds zonal stats to GDFs and plots results on a subplot each.

    The majority of hard coded user inputs are declared as enclosing scope variables here.

    Parameters
    ----------
    state_list : str or list
        Setting the default string value in the function definition causes this state to be active on-load. This is useful when service area source files are not available for all states (selecting such a state will cause and OpenFailedError). When interact() is executed, the list of states is passed to main().
    sw_ppd : tuple
        Floats for min, max and step controlling the ipywidgets slider
    """

    # USER INPUT 1: Enclosing Scope Variables

    # equates the element from the list passed to main() by interact() with state_select
    state_select = state_list

    # indicate the name of the column containing the names of municipalities
    mun_name_field = 'ADM2_EN'

    # indicating the filepath to the service areas data source(s)
    fp_service_areas = 'data_files/01_input/01_vector/service_areas_' + state_select.lower() + '.shp'

    # indicating the name of the column containing the names of service providers;
    provider_name_field = 'psp_name'

    # indicate the name of the column containing the weekly collection totals reported
    provider_coll_field = 'total_coll'

    # indicate the filepath to the raster data source;
    fp_raster = 'data_files/01_input/02_raster/hrsl_nga_pop.tif'

    # OPTIONAL CUSTOMIZATIONS

    # the rasterstats zonal statistic to be computed for each jurisdiction (default is 'sum')
    stat_select = 'sum' 

    # If adjusting 'stat_select', also adjust the title of the choropleth maps accordingly
    var_name = 'Tonnes of solid waste generated per week'
    
    # EXECUTE FUNCTIONS

    municipal_filter, bbox, service_areas = getVector(fp_adm, fp_service_areas, state_name_field, state_select)

    array, affine, nodata = computeArray(fp_raster, bbox, sw_ppd)

    mun_names, mun_stats, provider_names, provider_stats = zonalStats(municipal_filter, service_areas, array, affine, nodata, stat_select, mun_name_field, provider_name_field)

    mun_dict_sorted, provider_dict_sorted = processStats(service_areas, mun_names, mun_stats, provider_names, provider_stats, provider_coll_field)

    # UPDATE GEODATAFRAMES WITH RESULTS
    
    # Assign zonal statistics to new columns 
    # 'stat_output' for municipal_filter GDF and 'total_uncoll' for service_areas GDF
    # using two different methods
    
    municipal_filter = municipal_filter.assign(
            stat_output = pd.Series(mun_stats, index = municipal_filter.index)
    )

    for i, row in service_areas.iterrows():
            service_areas.loc[i, 'total_uncoll'] = provider_stats[i] - row[provider_coll_field]
    
    # PLOT RESULTS ON CHOROPLETH MAPS
    
    # Define figure CRS and canvas layout

    myCRS = ccrs.Mercator()

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7.5), subplot_kw=dict(projection=myCRS))

    # --Subplot 1-- 
    
    # add municipal boundaries

    municipal_feat = ShapelyFeature(municipal_filter['geometry'], myCRS, facecolor='none', edgecolor='k', linewidth=0.5)
    ax1.add_feature(municipal_feat)

    # add dynamic title and annotation

    title = var_name + ' by municipalities in ' + state_select + ' State'
    ax1.set_title(title, fontdict={'fontsize': '12', 'fontweight' : '5'})

    ax1.axis('off')

    ax1.annotate('Source: CIESIN HRSL, assuming ' + str(sw_ppd) + 'kg of solid waste per capita per day', xy=(0.225, .025), xycoords='figure fraction', fontsize=12, color='#555555')

    # create colorbar legend

    vmin, vmax =  municipal_filter['stat_output'].min(), municipal_filter['stat_output'].max()

    sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))

    sm.set_array([])

    fig.colorbar(sm, ax=ax1, orientation="horizontal")

    municipal_filter.plot(column='stat_output', cmap='Blues', linewidth=0.8, ax=ax1, edgecolor='0.8')

    # --Sub-plot 2--

    # add dynamic title and annotation

    title2 = 'Tonnes of uncollected solid waste per week by service area (dummy data)'
    ax2.set_title(title2, fontdict={'fontsize': '12', 'fontweight' : '5'})

    ax2.axis('off')

    # Create colorbar legend

    vmin2, vmax2 =  service_areas['total_uncoll'].min(), service_areas['total_uncoll'].max()

    sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin2, vmax=vmax2))

    sm.set_array([])

    fig.colorbar(sm, ax=ax2, orientation="horizontal")

    municipal_filter.plot(facecolor='none', linewidth=0.5, ax=ax2, edgecolor='k')

    service_areas.plot(column='total_uncoll', cmap='Reds', linewidth=0.8, ax=ax2, edgecolor='k')

#### Executing `main()` through `interact()`

Global scope variables are defined in the **USER INPUTS 2** section outside any function. This makes state_list accessible to `interact()` allowing the drop-down menu to be generated.

`main()` is called by the *ipywidget* `interact()` function to interactively manipulate the `state_list` variable for selecting study areas (passed to `getVector()` and the `sw_ppd` variable indicating the amount of solid waste generated per capita per day which is passed to the `computeArray()` sub-task function whenever the user adjusts the slider widget. 


In [18]:
# USER INPUTS 2: Global Scope Variables
# this set of variables is declared globally to make state_list accessible to interact()

fp_adm = 'data_files/01_input/01_vector/nga_admbnda_adm2_osgof_20190417.shp'

# indicate the name of the column indicating the names of the superordinate (e.g. state-level) jurisdictions
# If this information is in a data source separate from the subordinate tier,
# a spatial join via GeoPandas or a desktop GIS may be necessary

state_name_field = 'ADM1_EN'

# variables declared to be passed to interact()

municipal_all = gpd.read_file(fp_adm)

state_list = sorted(municipal_all[state_name_field].unique().tolist())

# EXECUTE

interact(main, state_list=state_list)

interactive(children=(Dropdown(description='state_list', index=24, options=('Abia', 'Adamawa', 'Akwa Ibom', 'A…

<function __main__.main(state_list='Lagos', sw_ppd=(0.4, 1.2, 0.1))>